In [1]:
import pandas as pd
from plotting import CandlePlot
pd.set_option("display.max_columns", None)

In [2]:
df_ma_res = pd.read_pickle("../data/ma_res.pkl")
df_ma_trades = pd.read_pickle("../data/ma_trades.pkl")

In [3]:
df_ma_res.granularity.unique()

array(['H1', 'H4'], dtype=object)

In [9]:
df_ma_res.head()

,pair,num_trades,total_gain,mean_gain,min_gain,max_gain,ma_l,ma_s,cross,granularity
0,EUR_USD,226,-1574,-6,-219,216,MA_20,MA_10,MA_10_MA_20,H1
1,EUR_USD,116,-815,-7,-147,263,MA_40,MA_10,MA_10_MA_40,H1
2,EUR_USD,111,-273,-2,-176,233,MA_40,MA_20,MA_20_MA_40,H1
3,EUR_USD,124,352,2,-190,260,MA_40,MA_30,MA_30_MA_40,H1
4,EUR_USD,80,-938,-11,-219,235,MA_80,MA_10,MA_10_MA_80,H1


In [4]:
df_ma_res_h1 = df_ma_res[df_ma_res.granularity == "H1"].copy()
df_ma_trades_h1 = df_ma_trades[df_ma_trades.granularity == "H1"].copy()

In [5]:
df_ma_trades_h1.shape

(17972, 32)

In [6]:
df_ma_res_h1.columns

Index(['pair', 'num_trades', 'total_gain', 'mean_gain', 'min_gain', 'max_gain',
       'ma_l', 'ma_s', 'cross', 'granularity'],
      dtype='object')

In [10]:
df_cross_summary = df_ma_res_h1[['pair', 'num_trades', 'total_gain','cross']].copy()

In [18]:
df_cross_gr = df_cross_summary.groupby(by="cross", as_index=False).sum(numeric_only=True)
df_cross_gr.sort_values(by="total_gain", ascending=False, inplace=True)

In [19]:
df_cross_gr.head()

,cross,num_trades,total_gain
15,MA_40_MA_80,820,-4737
11,MA_30_MA_40,1870,-6222
2,MA_10_MA_20,3198,-6280
4,MA_10_MA_80,1163,-8000
12,MA_30_MA_80,858,-8624


## find the best cross

In [14]:
total_pairs = len(df_ma_res_h1.pair.unique())

In [15]:
total_pairs

15

In [16]:
temp = df_ma_res_h1[df_ma_res_h1.cross == "MA_10_MA_40"]

In [20]:
temp

,pair,num_trades,total_gain,mean_gain,min_gain,max_gain,ma_l,ma_s,cross,granularity
1,EUR_USD,116,-815,-7,-147,263,MA_40,MA_10,MA_10_MA_40,H1
17,EUR_GBP,107,314,2,-106,245,MA_40,MA_10,MA_10_MA_40,H1
33,EUR_JPY,121,-1356,-11,-386,490,MA_40,MA_10,MA_10_MA_40,H1
49,EUR_AUD,139,-2698,-19,-225,287,MA_40,MA_10,MA_10_MA_40,H1
65,EUR_CAD,119,-682,-5,-152,253,MA_40,MA_10,MA_10_MA_40,H1
81,USD_JPY,122,-662,-5,-410,488,MA_40,MA_10,MA_10_MA_40,H1
97,USD_CAD,127,-1603,-12,-170,273,MA_40,MA_10,MA_10_MA_40,H1
113,GBP_USD,112,193,1,-224,553,MA_40,MA_10,MA_10_MA_40,H1
129,GBP_JPY,114,1128,9,-436,772,MA_40,MA_10,MA_10_MA_40,H1
145,GBP_AUD,141,-614,-4,-179,783,MA_40,MA_10,MA_10_MA_40,H1


In [22]:
temp[temp.total_gain > 0].shape[0] # number of pair that get positive gain

4

In [23]:
for c in df_ma_res_h1.cross.unique():
    temp = df_ma_res_h1[df_ma_res_h1.cross == c]
    pg = temp[temp.total_gain > 0].shape[0]
    print(f"{c} pg:{pg} perc gain:{(pg / total_pairs) * 100:.0f}%")

MA_10_MA_20 pg:4 perc gain:27%
MA_10_MA_40 pg:4 perc gain:27%
MA_20_MA_40 pg:3 perc gain:20%
MA_30_MA_40 pg:6 perc gain:40%
MA_10_MA_80 pg:6 perc gain:40%
MA_20_MA_80 pg:2 perc gain:13%
MA_30_MA_80 pg:5 perc gain:33%
MA_40_MA_80 pg:5 perc gain:33%
MA_10_MA_120 pg:3 perc gain:20%
MA_20_MA_120 pg:3 perc gain:20%
MA_30_MA_120 pg:3 perc gain:20%
MA_40_MA_120 pg:2 perc gain:13%
MA_10_MA_150 pg:2 perc gain:13%
MA_20_MA_150 pg:3 perc gain:20%
MA_30_MA_150 pg:2 perc gain:13%
MA_40_MA_150 pg:1 perc gain:7%


## how good is one choson cross

In [24]:
cross = "MA_10_MA_20"
df_MA_10_MA_20 = df_ma_res_h1[df_ma_res_h1.cross == cross].copy()
df_MA_10_MA_20.sort_values(by="total_gain", ascending=False, inplace=True)

In [25]:
df_MA_10_MA_20

,pair,num_trades,total_gain,mean_gain,min_gain,max_gain,ma_l,ma_s,cross,granularity
128,GBP_JPY,193,1266,6,-425,668,MA_20,MA_10,MA_10_MA_20,H1
160,GBP_CAD,216,1106,5,-194,543,MA_20,MA_10,MA_10_MA_20,H1
144,GBP_AUD,206,446,2,-244,727,MA_20,MA_10,MA_10_MA_20,H1
112,GBP_USD,208,146,0,-249,388,MA_20,MA_10,MA_10_MA_20,H1
208,AUD_CAD,209,-179,0,-129,200,MA_20,MA_10,MA_10_MA_20,H1
80,USD_JPY,209,-307,-1,-355,662,MA_20,MA_10,MA_10_MA_20,H1
224,CAD_JPY,216,-443,-2,-259,294,MA_20,MA_10,MA_10_MA_20,H1
32,EUR_JPY,202,-512,-2,-352,521,MA_20,MA_10,MA_10_MA_20,H1
192,AUD_JPY,207,-639,-3,-241,260,MA_20,MA_10,MA_10_MA_20,H1
64,EUR_CAD,212,-641,-3,-160,218,MA_20,MA_10,MA_10_MA_20,H1


In [26]:
df_gains = df_ma_trades_h1[(df_ma_trades_h1.cross==cross)&
                           (df_ma_trades_h1.pair=="CAD_JPY")].copy()

In [27]:
df_gains.head(3)

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,MA_40,MA_10,MA_80,MA_20,MA_150,MA_120,MA_30,DELTA,DELTA_PREV,TRADE,DIFF,GAIN,granularity,pair,GAIN_C,ma_l,ma_s,cross
16748,2022-08-22 13:00:00+00:00,7358,105.584,105.608,105.204,105.342,105.574,105.598,105.194,105.333,105.593,105.617,105.214,105.352,105.358775,105.4522,105.020363,105.46665,104.542733,104.643333,105.434133,-0.01445,0.01815,-1,0.199,-19.9,H1,CAD_JPY,-19.9,MA_20,MA_10,MA_10_MA_20
16749,2022-08-23 10:00:00+00:00,2774,105.508,105.590,105.451,105.541,105.497,105.580,105.442,105.532,105.518,105.600,105.460,105.550,105.401975,105.3483,105.204150,105.34570,104.669240,104.985492,105.365300,0.00260,-0.01710,1,-0.275,-27.5,H1,CAD_JPY,-47.4,MA_20,MA_10,MA_10_MA_20
16750,2022-08-24 05:00:00+00:00,2571,105.208,105.274,105.128,105.266,105.199,105.264,105.118,105.257,105.218,105.286,105.137,105.275,105.388125,105.4362,105.373450,105.44875,104.935827,105.142950,105.402633,-0.01255,0.01095,-1,0.390,-39.0,H1,CAD_JPY,-86.4,MA_20,MA_10,MA_10_MA_20


In [30]:
cp = CandlePlot(df_gains, candles=True)
cp.show_plot(line_traces=['GAIN_C'])

In [29]:
for curr in df_MA_10_MA_20[df_MA_10_MA_20.total_gain > 0].pair.unique():
    temp_gains = df_ma_trades_h1[(df_ma_trades_h1.cross==cross)&
                           (df_ma_trades_h1.pair==curr)].copy()
    print(curr, int(temp_gains.GAIN.sum()))
    cp = CandlePlot(temp_gains, candles=False)
    cp.show_plot(line_traces=['GAIN_C'])

GBP_JPY 1266


GBP_CAD 1106


GBP_AUD 446


GBP_USD 146
